<a href="https://colab.research.google.com/github/darkwingpatil/Ml_hackethons/blob/main/Keras_Autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Certification in AIML
## A Program by IIIT-H and TalentSprint

### Objective
- Perform denoising of the CIFAR-10 dataset images using a Convolutional Autoencoder (AE)

In [ ]:
#@title Experiment Explanation Video
from IPython.display import HTML

HTML("""<video width="850" height="480" controls>
  <source src="https://cdn.iiith.talentsprint.com/aiml/Experiment_related_data/Walkthrough/Denosing_CIFAR10_Walkthrough.mp4" type="video/mp4">
</video>
""")



### Dataset

In this experiment, we will use the CIFAR-10 dataset. It consists of 60,000 32x32 colour images in 10 classes, with 6000 images per class. There are 50,000 training images and 10,000 test images.

**Dataset:** CIFAR-10 is a small image (32 x 32) dataset made up of 60000 images subdivided into 10 main categories as shown below:


![alt text](https://debuggercafe.com/wp-content/uploads/2019/12/cifar10_small.jpg)

### AI/ML Technique

Autoencoders are neural networks that aim to copy their inputs to their outputs. They compress the input into a latent-space representation, and then reconstruct the output from this representation. This kind of network is composed of two parts :(1) Encoder - compresses the input into a latent-space representation (2) Decoder - reconstructs the input from the latent space representation. In training the autoencoder to copy the input to the output, the latent representation 'h' takes on useful properties. This leads to its wide application in dimension reduction and image noise reduction. The image below shows how images are reconstructed by an autoencoder
![alt text](https://miro.medium.com/max/600/1*V_YtxTFUqDrmmu2JqMZ-rA.png)

### Setup Steps:

In [ ]:
#@title Please enter your registration id to start: { run: "auto", display-mode: "form" }
Id = "" #@param {type:"string"}

In [ ]:
#@title Please enter your password (normally your phone number) to continue: { run: "auto", display-mode: "form" }
password = "" #@param {type:"string"}

In [ ]:
#@title Run this cell to complete the setup for this Notebook
from IPython import get_ipython
import re
ipython = get_ipython()

notebook= "U3W14_52_Denoising_Autoencoder_CIFAR10_C" #name of the notebook

def setup():
#  ipython.magic("sx pip3 install torch")
    from IPython.display import HTML, display
    display(HTML('<script src="https://dashboard.talentsprint.com/aiml/record_ip.html?traineeId={0}&recordId={1}"></script>'.format(getId(),submission_id)))
    print("Setup completed successfully")
    return

def submit_notebook():
    ipython.magic("notebook -e "+ notebook + ".ipynb")

    import requests, json, base64, datetime

    url = "https://dashboard.talentsprint.com/xp/app/save_notebook_attempts"
    if not submission_id:
      data = {"id" : getId(), "notebook" : notebook, "mobile" : getPassword()}
      r = requests.post(url, data = data)
      r = json.loads(r.text)

      if r["status"] == "Success":
          return r["record_id"]
      elif "err" in r:
        print(r["err"])
        return None
      else:
        print ("Something is wrong, the notebook will not be submitted for grading")
        return None

    elif getAnswer() and getComplexity() and getAdditional() and getConcepts() and getWalkthrough() and getComments() and getMentorSupport():
      f = open(notebook + ".ipynb", "rb")
      file_hash = base64.b64encode(f.read())

      data = {"complexity" : Complexity, "additional" :Additional,
              "concepts" : Concepts, "record_id" : submission_id,
              "answer" : Answer, "id" : Id, "file_hash" : file_hash,
              "notebook" : notebook, "feedback_walkthrough":Walkthrough ,
              "feedback_experiments_input" : Comments,
              "feedback_inclass_mentor": Mentor_support}

      r = requests.post(url, data = data)
      r = json.loads(r.text)
      if "err" in r:
        print(r["err"])
        return None
      else:
        print("Your submission is successful.")
        print("Ref Id:", submission_id)
        print("Date of submission: ", r["date"])
        print("Time of submission: ", r["time"])
        print("View your submissions: https://aiml-iiith.talentsprint.com/notebook_submissions")
        #print("For any queries/discrepancies, please connect with mentors through the chat icon in LMS dashboard.")
        return submission_id
    else: submission_id


def getAdditional():
  try:
    if not Additional:
      raise NameError
    else:
      return Additional
  except NameError:
    print ("Please answer Additional Question")
    return None

def getComplexity():
  try:
    if not Complexity:
      raise NameError
    else:
      return Complexity
  except NameError:
    print ("Please answer Complexity Question")
    return None

def getConcepts():
  try:
    if not Concepts:
      raise NameError
    else:
      return Concepts
  except NameError:
    print ("Please answer Concepts Question")
    return None


def getWalkthrough():
  try:
    if not Walkthrough:
      raise NameError
    else:
      return Walkthrough
  except NameError:
    print ("Please answer Walkthrough Question")
    return None

def getComments():
  try:
    if not Comments:
      raise NameError
    else:
      return Comments
  except NameError:
    print ("Please answer Comments Question")
    return None


def getMentorSupport():
  try:
    if not Mentor_support:
      raise NameError
    else:
      return Mentor_support
  except NameError:
    print ("Please answer Mentor support Question")
    return None

def getAnswer():
  try:
    if not Answer:
      raise NameError
    else:
      return Answer
  except NameError:
    print ("Please answer Question")
    return None


def getId():
  try:
    return Id if Id else None
  except NameError:
    return None

def getPassword():
  try:
    return password if password else None
  except NameError:
    return None

submission_id = None
### Setup
if getPassword() and getId():
  submission_id = submit_notebook()
  if submission_id:
    setup()
else:
  print ("Please complete Id and Password cells before running setup")



### Importing required packages

In [ ]:
from keras.models import Sequential
from keras.layers import Input, Conv2D, UpSampling2D, MaxPooling2D
from keras.datasets import cifar10

import numpy as np
import matplotlib.pyplot as plt

### Load the Data

In [ ]:
# Loading the data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

### Normalize the data within the range of 0 and 1

In [ ]:
# Normalize the train and test coloured images
x_train = x_train/255
x_test = x_test/255

### Build and compile the autoencoder model

**padding:** In Keras, it is specified via the “padding” argument on the Conv2D layer, which has the default value of ‘valid‘ (no padding). This means that the filter is applied only to valid ways of the input.

The ‘padding‘ value of ‘same‘ calculates and adds the padding required to the input image (or feature map) to ensure that the output has the same shape as the input.

**Upsampling:** The Upsampling layer is a simple layer with no weights that will double the dimensions of input and can be used in a generative model when followed by a traditional convolutional layer

In [ ]:
model = Sequential()

model.add(Conv2D(32, kernel_size=3, padding='same', activation='relu', input_shape=(32, 32, 3))) # 32x32x32
model.add(MaxPooling2D(2, 2))    # 16x16x32
model.add(Conv2D(32, kernel_size=3, padding='same', activation='relu'))      # 16x16x32
model.add(MaxPooling2D(2, 2))     # 8x8x32

model.add(Conv2D(32, kernel_size=3, padding='same', activation='relu'))      # 8x8x32
# To double the dimensions of input
model.add(UpSampling2D((2, 2))) # 16x16x32
model.add(Conv2D(32, kernel_size=3, padding='same', activation='relu'))      # 16x16x32
model.add(UpSampling2D((2, 2))) # 32x32x32
model.add(Conv2D(3,  kernel_size=3, padding='same', activation='sigmoid'))   # 32x32x3

model.compile(optimizer='adam', metrics=['accuracy'], loss='mean_squared_error')
model.summary()

### Training the model

Training the model by fitting the noisy images and original images with different noise

In [ ]:
# Adding different noise vectors for each epoch
num_epochs = 3    # It takes some time to run this cell

NOISE = 0.3
for i in range(num_epochs):
    noise = np.random.normal(0, NOISE, x_train.shape)
    model.fit(x_train + noise, x_train, batch_size=100)

### Testing the model with 400 images

In [ ]:
# Test data with 400 images
x_test = x_test[:400]

# Taking noise randomly
noise = np.random.normal(0, NOISE, x_test.shape)

# Adding noise to the test data
noisy = x_test + noise

# Predicting the test images
pred_imgs = model.predict(noisy)

### Comparative visualization of Noisy Input Images and Denoised Reconstructed output



Below function takes the input of test images and to plot the images in subplot by taking unique label indices

In [ ]:
def plot_Images(test_Images):
    image=1
    plt.figure(figsize=(15,12))
    for i in np.unique(y_test):
      idx = np.where(y_test==i)[0][0]
      plt.subplot(1,10,image)
      Image = np.clip(test_Images[idx], 0, 1)
      plt.imshow(Image)
      image +=1
    plt.show()

In [ ]:
print("Original Images \n")
plot_Images(x_test)

In [ ]:
print("Images with noise \n")
plot_Images(noisy)

In [ ]:
print("Images after decoding \n")
plot_Images(pred_imgs)

### Please answer the questions below to complete the experiment:




In [ ]:
#@title State True or False: During the training, Autoencoders will be able to remove noise from the output images without comparing the original images { run: "auto", form-width: "500px", display-mode: "form" }
Answer = "" #@param ["","TRUE", "FALSE"]


In [ ]:
#@title How was the experiment? { run: "auto", form-width: "500px", display-mode: "form" }
Complexity = "" #@param ["","Too Simple, I am wasting time", "Good, But Not Challenging for me", "Good and Challenging for me", "Was Tough, but I did it", "Too Difficult for me"]


In [ ]:
#@title If it was too easy, what more would you have liked to be added? If it was very difficult, what would you have liked to have been removed? { run: "auto", display-mode: "form" }
Additional = "" #@param {type:"string"}


In [ ]:
#@title Can you identify the concepts from the lecture which this experiment covered? { run: "auto", vertical-output: true, display-mode: "form" }
Concepts = "" #@param ["","Yes", "No"]


In [ ]:
#@title  Experiment walkthrough video? { run: "auto", vertical-output: true, display-mode: "form" }
Walkthrough = "" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [ ]:
#@title  Text and image description/explanation and code comments within the experiment: { run: "auto", vertical-output: true, display-mode: "form" }
Comments = "" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [ ]:
#@title Mentor Support: { run: "auto", vertical-output: true, display-mode: "form" }
Mentor_support = "" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [ ]:
#@title Run this cell to submit your notebook for grading { vertical-output: true }
try:
  if submission_id:
      return_id = submit_notebook()
      if return_id : submission_id = return_id
  else:
      print("Please complete the setup first.")
except NameError:
  print ("Please complete the setup first.")